## Install packages

In [ ]:
## install langchain
!pip install langchain openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available and initialized.")
else:
    print("GPU is not available or not properly initialized.")


GPU is available and initialized.


In [ ]:

from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from langchain.docstore.document import Document


## generate content chain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


In [ ]:
# # Generate the context using the question and answer
# context = f"Question: {question}\nAnswer: {answer}"

# Define the prompt template
prompt_template = """
Use the following pieces of context, question, generate content to fintune bert model dont' be creative please ensure to include the context

Example :

question: "What are the advantages and disadvantages of PCA and Autoencoder for feature extraction?"

context: "PCA is computationally efficient and easy to interpret, but it may not capture complex non-linear patterns in the data. On the other hand, an Autoencoder can capture non-linear relationships but may require more computational resources for training."

content: "Feature extraction is an essential step in unsupervised learning. There are different methods available for extracting features, such as PCA (Principal Component Analysis) and Autoencoder. PCA is a linear dimensionality reduction technique that aims to find the orthogonal axes with maximum variance in the data. On the other hand, an Autoencoder is a neural network that learns to encode and decode data, capturing non-linear relationships in the process. Both methods have their advantages and disadvantages. PCA is computationally efficient and easy to interpret but may not capture complex non-linear patterns in the data. Autoencoders, on the other hand, can capture non-linear relationships but may require more computational resources for training. So, the choice between PCA and Autoencoder depends on the specific requirements of the task at hand."


{context}
Question: {question}


content:"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context",  "question"]
)

In [ ]:
# Load the question-answering chain
generate_chain = load_qa_chain(
    OpenAI(temperature=0.7,model="text-davinci-003", openai_api_key=openai_api_key ,  max_tokens=2000),
    chain_type="stuff",
    output_key="content",
    prompt=PROMPT
)

# Define the question and answer
question = "Which probability distribution is commonly used for modelling a multi-class classification task?"
answer = "It is common to model a multi-class classification task with a model that predicts a Multinoulli probability distribution."
# Convert the answer into a Document object
answer_doc = Document(page_content=answer, text=answer)

# Provide the input data (question, answer, and initial context)
input_data = {
    "question": question,
    "input_documents": [answer_doc],
    "answer": ""
}


In [ ]:
generate_chain({"input_documents": [answer_doc], "question": question}, return_only_outputs=True)


{'content': ' A multi-class classification task involves predicting a categorical outcome with multiple possible classes. To model such a task, a probability distribution is commonly used to represent the likelihood of each class. The most commonly used probability distribution for this purpose is the Multinoulli distribution. This distribution assigns a probability to each class and can be used to model the probability of each class. The Multinoulli distribution is suitable for modelling multi-class classification tasks as it can capture the probability of each class and the relationships between them.'}

## validate answer chain

In [ ]:

# Define the prompt template
prompt_template =  """

Use the following pieces of context to answer the question at the end. Please extract the answer exactly as it appears in the context without any modifications or addition and make it valid to fintune bert model .

Example :
'context': 'Beyoncé Giselle Knowles-Carter  (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
'question': 'When did Beyonce start becoming popular?',
'answers': 'text': ['in the late 1990s'],
'answer_start': [269]


Context:
{context}

Question: {question}

Answer:
"""

# Your code to extract the start position of the answer goes here
query = "what is genetic programming"
docs = "Genetic Programming (GP) is a type of evolutionary algorithm used in Machine Learning. It applies evolutionary principles, such as selection, crossover, and mutation, to automatically generate computer programs that have the desired characteristics. GP involves creating a population of computer programs, which are then tested and evaluated based on their ability to solve a given problem. The best-performing programs are then selected and used to create a new population. This process is repeated until the desired objective is met. Genetic Programming has been used in a range of areas, such as image recognition, natural language processing, and optimization."
docs = Document(page_content=docs, text=docs)

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
validate_chain = load_qa_chain(
    OpenAI(temperature=0.5,model="text-davinci-003", openai_api_key=openai_api_key ,  max_tokens=2000),
    chain_type="stuff",
    output_key="review",
    prompt=PROMPT,
)



In [ ]:
validate_chain({"input_documents": [docs], "question": query})

{'input_documents': [Document(lc_kwargs={'page_content': 'Genetic Programming (GP) is a type of evolutionary algorithm used in Machine Learning. It applies evolutionary principles, such as selection, crossover, and mutation, to automatically generate computer programs that have the desired characteristics. GP involves creating a population of computer programs, which are then tested and evaluated based on their ability to solve a given problem. The best-performing programs are then selected and used to create a new population. This process is repeated until the desired objective is met. Genetic Programming has been used in a range of areas, such as image recognition, natural language processing, and optimization.', 'text': 'Genetic Programming (GP) is a type of evolutionary algorithm used in Machine Learning. It applies evolutionary principles, such as selection, crossover, and mutation, to automatically generate computer programs that have the desired characteristics. GP involves crea

## Sequential Chain

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

In [ ]:

overall_chain = SequentialChain(
    chains=[generat_chaine, validat_chain],
    input_variables=["input_documents", "question"],
    # Here we return multiple variables
    output_variables=["content", "review"],
    verbose=True)

In [ ]:
# Define the question and answer
question = "what is genetic programming"
answer = "Genetic programming is one of the two techniques used in machine learning. The model is based on the testing and selecting the best choice among a set of results."
# Convert the answer into a Document object
answer_doc = Document(page_content=answer, text=answer)

In [ ]:
result = overall_chain({"input_documents":[answer_doc], "question": question})



> Finished chain.


In [ ]:
result

{'input_documents': [Document(lc_kwargs={'page_content': 'Genetic programming is one of the two techniques used in machine learning. The model is based on the testing and selecting the best choice among a set of results.', 'text': 'Genetic programming is one of the two techniques used in machine learning. The model is based on the testing and selecting the best choice among a set of results.'}, page_content='Genetic programming is one of the two techniques used in machine learning. The model is based on the testing and selecting the best choice among a set of results.', metadata={})],
 'question': 'what is genetic programming',
 'content': ' Genetic Programming is a type of machine learning technique that employs genetic algorithms to optimize a problem by evolving a set of possible solutions. It works by creating a population of possible solutions, called individuals, and then testing them through a fitness function. The individuals with the highest fitness scores are then selected to

In [ ]:
result["content"]

' The most commonly used probability distribution for modelling a multi-class classification task is the Multinoulli distribution. This distribution is used to model the probability of a set of mutually exclusive outcomes, i.e. the probability of each class in a multi-class classification task. The Multinoulli distribution is a generalization of the Bernoulli distribution to more than two outcomes, and is easy to interpret and computationally efficient.'

## chaine pipline

In [ ]:
def pipeline_chain(question, answer):
    """
    Runs a pipeline of operations to generate content and validate the answer.

    Args:
        question (str): The question.
        answer (str): The answer.

    Returns:
        dict: A dictionary containing the question, generated content, and answer review.
    """
    answer_doc = Document(page_content=answer, text=answer)
    content = generate_chain({"input_documents": [answer_doc], "question": question}, return_only_outputs=True)
    docs = Document(page_content=content["content"], text=content["content"])
    result = validate_chain({"input_documents": [docs], "question": question})

    return {
        "question": question,
        "content": content["content"],
        "answer": result["review"]
    }


In [ ]:

question = "what is genetic programming"
answer = "Genetic programming is one of the two techniques used in machine learning. The model is based on the testing and selecting the best choice among a set of results."

pipeline_chain(question ,answer)

{'question': 'what is genetic programming',
 'content': ' Genetic Programming is an evolutionary computation technique, which involves using genetic algorithms to evolve a set of rules or programs to solve a problem. By defining a set of parameters and then allowing a computer to optimize them, it creates a population of programs that it evaluates and then selects the best one for further optimization. This process of selection and optimization is repeated until an optimal solution is found. Genetic Programming is used in a variety of applications, from machine learning to natural language processing, and can be used to solve complex problems that are difficult to solve using traditional methods.',
 'answer': "'text': ['Genetic Programming is an evolutionary computation technique, which involves using genetic algorithms to evolve a set of rules or programs to solve a problem'], \n'answer_start': [0]"}